#### 期货市场行情与基本面数据

期货数据可以从新浪财经数据接口获取，基本面数据可以通过通联数据接口获取。

In [1]:
import pandas as pd
import json
import requests
import pyecharts
import matplotlib.pyplot as plt  
import datetime as dt
import ssl
from pyecharts import Line
ssl._create_default_https_context = ssl._create_unverified_context
#中文字体显示  
plt.rc('font', family='SimHei', size=13)
%matplotlib inline

In [ ]:
#新浪财经的实时价格数据
def future_quote_sina(exchange='all', flag=1):
    '''获取新浪期货全部合约即时行情数据,默认取全部交易所主力合约，取特定交易所主力或全部合约，用参数来指定
    para:
        exchange:   all,shfe,dce,czce
        flag:       1（active contracts),0(non-active contracts),2(all contracts)
    result:
        dataframe
    '''
    r = requests.get(
        'http://money.finance.sina.com.cn/d/api/openapi_proxy.php/?__s=[[%22qhhq%22,%22qbhy%22,%22zdf%22,1000]]')
    data = r.text
    data = json.loads(data)
    d = data[0]
    df = pd.DataFrame(d['items'], columns=d['fields'])
    df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))  #
    df.replace({'name':{'\d+':''}},regex=True,inplace=True)
    # 交易所主力合约：df[(df.market=='dce')& (df.is_hot=='1')]
    # 全部主力合约：df[(df.market=='all')& (df.is_hot=='1')]
    # 交易所合约：df[(df.market=='dce')]
    df.set_index('symbol',inplace=True)
    if exchange == 'all':
        if flag == 0:
            # flag=0,返回四大交易所非主力合约
            return df[df.is_hot == flag].sort_values('volume', ascending=False)
        elif flag==1:
            #flag=1,返回四大交易所主力合约
            return df[df.is_hot >= flag].sort_values('volume', ascending=False)
        elif flag==-1:
            # flag=-1,返回四大交易所全部合约
            return df.sort_values('volume', ascending=False)
    else:
            # 指定交易所主力（非主力）合约
        if flag == 0:
            #返回指定交易所非主力合约
            return df[(df.market == exchange) & (df.is_hot == flag)].sort_values('volume', ascending=False)
        elif flag == 1:
            #返回指定交易所非主力合约
            return df[(df.market == exchange) & (df.is_hot >= flag)].sort_values('volume', ascending=False)
        else:
            # 指定交易所全部合约
            return df[df.market == exchange].sort_values('volume', ascending=False)

f=future_quote_sina('all',-1)


In [ ]:
def tongLianData(indicId=2020100024,start=20100101,end=20171231):
    url='https://gw.wmcloud.com/rrp/mobile/whitelist/data?indicID=%s&beginDate=%s&endDate=%s'%(indicId,start,end)
    print(url)
    f=pd.read_json(url)
    d=pd.DataFrame(f.loc['dataDetail','dataDetailNewList'])
    d.columns=['data','date']
    d.index=d.date.map(lambda x: dt.datetime.fromtimestamp(x/1000))
    del d['date']
    d.sort_index(inplace=True)
    data=Line()
    attr=list(d.index.to_datetime().strftime('%Y%m%d'))
    price=list(d.data)
    data.add(f.loc['name','dataDetailNewList'],attr,price)
    return data.render_notebook()
    
def main():
    from tkinter import Tk
    r = Tk()
    # read the clipboard
    c = r.clipboard_get()
    # print(c)
    return tongLianData(c)

In [ ]:
main()

In [ ]:
import re
from urllib import parse
files=open(r"C:\Users\Administrator\Desktop\report.txt").readlines()
def download_report(url,title,doctype):
    path=r'D:\aliyun\Work\zjt\%s' %title
    print(path)
    response = requests.get(url)
    with open(path+'.'+doctype, 'wb') as f:
        f.write(response.content)

for raw_url in files:
    url=raw_url.strip('\n')
    url=parse.unquote(str(url))
    print(url)
    title=re.search('(?<=bulletin/).*?(?=\.)',str(url)).group()
    doctype=re.search('(?<=\.)\w{3}?(?=\?)',str(url)).group()
    print(title)
    try:
        download_report(url,title,doctype)
    except:
        print("error:%s\n%s" %(url))

In [ ]:
url='http://snap.windin.com/bulletin/%E5%91%A8%E6%9C%9F%E4%B9%8B%E8%BD%AE%EF%BC%8814%EF%BC%89%EF%BC%9A%E5%91%A8%E6%9C%9F%E8%BD%AC%E6%8D%A2%EF%BC%9A%E4%BC%B0%E5%80%BC%E5%BC%B9%E6%80%A7%E4%B8%8E%E4%BB%B7%E6%A0%BC%E5%BC%B9%E6%80%A7.pdf?rppsid=45156382&mediatype=16&pkid=383924815'

In [ ]:
title='周期之轮（14）：周期转换：估值弹性与价格弹性'

In [ ]:
download_report(url,title,'pdf')


In [ ]:
import requests
headers={'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}
resp = requests.head(r'https://www.python.org/',headers=headers)

status = resp.status_code
last_modified = resp.headers['last-modified']
content_type = resp.headers['content-type']
content_length = resp.headers['content-length']
resp.text

In [ ]:
content_type

In [ ]:
from socketserver import BaseRequestHandler, UDPServer
import time

class TimeHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        # Get message and client socket
        msg, sock = self.request
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), self.client_address)

if __name__ == '__main__':
    serv = UDPServer(('', 20000), TimeHandler)
    serv.serve_forever()

Got connection from ('127.0.0.1', 55341)


In [1]:
import time

_hello_resp = '''\
<html>
  <head>
     <title>Hello {name}</title>
   </head>
   <body>
     <h1>Hello {name}!</h1>
   </body>
</html>'''

def hello_world(environ, start_response):
    start_response('200 OK', [ ('Content-type','text/html')])
    params = environ['params']
    resp = _hello_resp.format(name=params.get('name'))
    yield resp.encode('utf-8')

_localtime_resp = '''\
<?xml version="1.0"?>
<time>
  <year>{t.tm_year}</year>
  <month>{t.tm_mon}</month>
  <day>{t.tm_mday}</day>
  <hour>{t.tm_hour}</hour>
  <minute>{t.tm_min}</minute>
  <second>{t.tm_sec}</second>
</time>'''

def localtime(environ, start_response):
    start_response('200 OK', [ ('Content-type', 'application/xml') ])
    resp = _localtime_resp.format(t=time.localtime())
    yield resp.encode('utf-8')

if __name__ == '__main__':
    from resty import PathDispatcher
    from wsgiref.simple_server import make_server

    # Create the dispatcher and register functions
    dispatcher = PathDispatcher()
    dispatcher.register('GET', '/hello', hello_world)
    dispatcher.register('GET', '/localtime', localtime)

    # Launch a basic server
    httpd = make_server('', 8080, dispatcher)
    print('Serving on port 8080...')
    httpd.serve_forever()

ModuleNotFoundError: No module named 'resty'

In [4]:
import os
names = os.listdir(r'c:\Users\Administrator\Desktop')

In [5]:
names

['Axure RP.lnk',
 'desktop.ini',
 'Fiddler 4.lnk',
 'Foxmail.lnk',
 'PyCharm.lnk',
 'report.txt',
 'Sublime Text 3.lnk',
 'Total Commander 64 bit.lnk',
 '~$财富与同花顺交易客户端功能详细对比.docx',
 '~WRL0005.tmp',
 '东方财富.lnk',
 '交易软件',
 '功能搜索',
 '功能搜索.txt',
 '新股申购.rp',
 '现货黄金_wind.png',
 '赢顺通用版.lnk']

In [6]:
pyfiles = [name for name in os.listdir(r'c:\Users\Administrator\Desktop')
            if name.endswith('.lnk')]

In [7]:
pyfiles

['Axure RP.lnk',
 'Fiddler 4.lnk',
 'Foxmail.lnk',
 'PyCharm.lnk',
 'Sublime Text 3.lnk',
 'Total Commander 64 bit.lnk',
 '东方财富.lnk',
 '赢顺通用版.lnk']

In [12]:

import glob
pyfiles = glob.glob(r'd:\aliyun\github\flask\docs\*.rst')

In [1]:
def add(x:int, y:int) -> int:
    return x + y

In [2]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



In [3]:
add.__annotations__

{'return': int, 'x': int, 'y': int}

In [5]:
def myfun():
    return 1, 2, 3

In [6]:
myfun()

(1, 2, 3)

In [7]:
a, b, c = myfun()

In [8]:
a

1

In [9]:
b

2

In [10]:
c

3